# SMPC Trade Demo - Data Scientist

# Step 1: Login to new Domains and Find Data

In [1]:

nl = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)
usa = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8082)
it = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8083)

In [ ]:
nl.datasets

In [ ]:
usa.datasets

In [2]:
it.datasets

In [ ]:
nl_data = nl.datasets['feb2020']
usa_data = usa.datasets['feb2020']
it_data = it.datasets['feb2020']

# Step 2: Local Computation

In [ ]:
nl_sum = nl_data.sum(0)
usa_sum = usa_data.sum(0)
it_sum = it_data.sum(0)

# Step 3: Cross-node Computation

In [3]:
# automatically converts tensors to SMPC tensors under the hood
# and then runs the sum across them
result = nl_sum + usa_sum + it_sum

# Step 4: Request Ability to Download Result

In [4]:
#fails
result.get()

In [5]:
# works - creates one request on each shareholder
result.request(reason="I'd like to decrypt this result")

In [ ]:
result.get()